# RedPanda Examples

In [1]:
import pandas, redpanda, sqlalchemy, sys
print "RedPanda version:", redpanda.__version__
print "Python version:", sys.version

RedPanda version: 0.2.0
Python version: 2.7.12 (default, Jun 29 2016, 08:57:23) 
[GCC 5.3.0]


## Opening a session

Open a `RedPanda` session to begin exploring data:

In [2]:
engine = redpanda.create_engine("sqlite://")
session = redpanda.Session(bind=engine)

Note that this is syntactic sugar for SQLAlchemy calls:

In [3]:
engine = sqlalchemy.create_engine("sqlite://")
sessionmaker = sqlalchemy.orm.sessionmaker(query_cls=redpanda.orm.Query, class_=redpanda.orm._Session)
session = sessionmaker(bind=engine)

## Creating An Example Model

Define your models using the declarative API. `RedPanda` defines one such model in the `example` module:

In [4]:
import sqlalchemy.ext.declarative
Base = sqlalchemy.ext.declarative.declarative_base()
class Widget(Base):
    id        = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    timestamp = sqlalchemy.Column(sqlalchemy.DateTime)
    name      = sqlalchemy.Column(sqlalchemy.String)
    kind      = sqlalchemy.Column(sqlalchemy.String)
    units     = sqlalchemy.Column(sqlalchemy.Integer)

    __read_sql__  = {"index_col": ["timestamp"], "parse_dates": ["timestamp"]}
    __tablename__ = "widgets"

Defining the class attribute `__read_sql__` will configure default arguments passed to `pandas.read_sql()`. 

In this case we are configuring the `widgets` table to use the `timestamp` column as an index and that we wish for `pandas` to parse this column as a `datetime` object.

In [5]:
from redpanda.example import create_widgets
create_widgets(session)

## Querying Data

Use the `frame()` method of a query to return the results as a `DataFrame`:

In [6]:
session.query(Widget).frame().head()

,id,name,kind,units
timestamp,,,,
2016-01-07 00:32:00,1,benches,buzzer,83
2016-01-07 14:25:00,2,endeavors,fizzer,25
2016-01-12 05:58:00,3,polices,fizzer,63
2016-01-17 19:34:00,4,readers,buzzer,34
2016-01-24 02:37:00,5,apostrophe,buzzer,13


### Querying with Filters

Use SQLAlchemy filters to refine the query:

In [7]:
session.query(Widget).filter(Widget.timestamp.between("2016-11-01", "2016-11-30")).frame()

,id,name,kind,units
timestamp,,,,
2016-11-11 04:49:00,61,backup,buzzer,6
2016-11-19 12:03:00,62,captains,fizzer,76
2016-11-23 19:13:00,63,forties,buzzer,57
2016-11-28 04:00:00,64,challenge,fizzer,66
2016-11-29 04:21:00,65,majors,buzzer,96


### Overriding Defaults

Pass arguments to `pandas.read_sql()` into the `frame()` method:

In [8]:
session.query(Widget).frame(index_col="id").head()

,timestamp,name,kind,units
id,,,,
1,2016-01-07 00:32:00,benches,buzzer,83
2,2016-01-07 14:25:00,endeavors,fizzer,25
3,2016-01-12 05:58:00,polices,fizzer,63
4,2016-01-17 19:34:00,readers,buzzer,34
5,2016-01-24 02:37:00,apostrophe,buzzer,13


## Writing DataFrame to Session

Use the `add_dataframe()` method of the `RedPanda` session to place `DataFrame` rows into the session as models:

In [9]:
from datetime import datetime
dfdata = {
    "kind": "newkind", 
    "name": "newname", 
    "units": 1000, 
    "timestamp": datetime.utcnow()}
addme = pandas.DataFrame(dfdata, index=[0])
addme

,kind,name,timestamp,units
0,newkind,newname,2016-07-05 16:38:59.757119,1000


In [10]:
session.add_dataframe(Widget, addme)
session.commit()
session.query(Widget).filter(Widget.kind=="newkind").frame()

,id,name,kind,units
timestamp,,,,
2016-07-05 16:38:59.757119,76,newname,newkind,1000
